In [ ]:
%matplotlib widget
from pathlib import Path
from shutil import copy

import flammkuchen as fl
import numpy as np
import pandas as pd
from lotr import LotrExperiment, dataset_folders
from matplotlib import pyplot as plt
from tqdm import tqdm

In [ ]:
coords_loc = Path("/Volumes/Shared/experiments/E0071_lotr/full_ring/")

In [ ]:
coords_loc.exists()

In [ ]:
for path in tqdm(dataset_folders[-4:]):
    src = coords_loc / path.parent.name / path.name / "transformaed_coords.h5"
    copy(src, path)

In [ ]:
path = dataset_folders[0]
exp = LotrExperiment(path)

In [ ]:
coords_df = []
for path in tqdm(dataset_folders):
    exp = LotrExperiment(path)
    coords = exp.coords_um
    cent_coords = coords - np.mean(coords, 0)
    try:
        morph_coords = exp.morphed_coords
    except OSError:
        morph_coords = np.full(coords.shape, np.nan)
    data_dict = {f"raw{i}": coords[:, i] for i in range(3)}
    data_dict.update({f"morphed{i}": morph_coords[:, i] for i in range(3)})
    data_dict.update({f"centered{i}": cent_coords[:, i] for i in range(3)})
    data_dict["fid"] = path.name

    coords_df.append(pd.DataFrame(data_dict))

coords_df = pd.concat(coords_df, ignore_index=1)

In [ ]:
valid_coords_df = coords_df[~np.isnan(coords_df["morphed0"])]

In [ ]:
f, axs = plt.subplots(1, 3, figsize=(7, 3))
for i, k in enumerate(["raw", "morphed", "centered"]):
    axs[i].scatter(
        valid_coords_df[k + "1"],
        -valid_coords_df[k + "0"],
        lw=0,
        s=5,
        alpha=0.03,
        color=(0.3,) * 3,
    )
    axs[i].set_title(k)
    axs[i].axis("equal")
    axs[i].axis("off")